## Примечание

Проект выполнен с помощью модели BERT, обученной на разговорном англоязычном корпусе. Данная модель была взята из открытого репозитория **DeepPavlov**. Так как проект носит учебный характер, а также в целях минимизации времени создания эмбеддингов, из выборки взяты только 400 случайных элементов. Для ускорений вычислений и использования библиотек PyTorch, Transformers проект был выполнен в облачной среде Google Colab.

## Подготовка данных

In [100]:
import torch
import transformers

In [101]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import re
from tqdm import notebook
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import f1_score,roc_auc_score

In [10]:
df = pd.read_csv('toxic_comments.csv')

In [11]:
df_main = df.sample(400).reset_index(drop=True)

Выведем на экран первые десять строк таблицы:

In [12]:
df_main.head(10)

,text,toxic
0,"""\n\nQUEEN (BAND)\n\nI saw your message (the s...",0
1,"As you know, only in cases of blatant vandalis...",0
2,Please disambiguation ECN \n\nin the IPv4#Head...,0
3,"""\n\n Please stop. If you continue to vandaliz...",0
4,"I see your point and agree, the previous image...",0
5,re: what if im gay \n\nfuck you fag i expect t...,1
6,"""\nNow it's even worse! How do you want it, sh...",0
7,"""Welcome!\n\nHello, and welcome to Wikipedia! ...",0
8,and Today Tonight a current affairs show host,0
9,"I solved it, didn't close the ref.",0


Получим общую информацию о таблице:

In [13]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    400 non-null    object
 1   toxic   400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


**В таблице 2 признака. Столбец `text` содержит текст комментария. Целевой признак, который нужно предсказать, — `toxic`.**

In [14]:
# очистим текст от лишних символов
def text_cleaner(text):
    clear_text = re.sub(r'[^a-zA-Z]', ' ', text).split()
    new_text = ' '.join(clear_text)
    return new_text

In [15]:
df_main['text'] = df_main['text'].apply(lambda x: text_cleaner(x))

In [16]:
df_main.head(10)

,text,toxic
0,QUEEN BAND I saw your message the second virtu...,0
1,As you know only in cases of blatant vandalism...,0
2,Please disambiguation ECN in the IPv Header di...,0
3,Please stop If you continue to vandalize Wikip...,0
4,I see your point and agree the previous image ...,0
5,re what if im gay fuck you fag i expect this v...,1
6,Now it s even worse How do you want it show me...,0
7,Welcome Hello and welcome to Wikipedia Thank y...,0
8,and Today Tonight a current affairs show host,0
9,I solved it didn t close the ref,0


Проведем предобработку данных с помощью модели BERT.

In [17]:
tokenizer = transformers.BertTokenizer( # инициализируем токенизатор
    vocab_file='vocab.txt')

In [18]:
tokenized = df_main['text'].apply( # преобразуем текст в номера токенов из словаря методом encode()
    lambda x: tokenizer.encode(x, max_length=512, truncation=True, add_special_tokens=True))

In [19]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

In [20]:
# применим метод padding, чтобы после токенизации длины исходных текстов в корпусе были равными
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

In [21]:
attention_mask = np.where(padded != 0, 1, 0) # создание «маски»
print(attention_mask.shape)

(400, 512)


In [22]:
# инициализируем модель класса BertModel и передадим ей файл с предобученной моделью и конфигурацией
config = transformers.BertConfig.from_json_file(
    'bert_config.json')
model = transformers.BertModel.from_pretrained(
    'pytorch_model.bin', config=config)

Some weights of the model checkpoint at gdrive/My Drive/Colab Notebooks/Project/Conversational BERT, English/pytorch_model.bin were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# создание эмбеддингов
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
features = np.concatenate(embeddings)

In [25]:
target = df_main['toxic']

In [70]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                            target, 
                                                                            test_size=0.25, 
                                                                            random_state=12345)

In [71]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(300, 768)
(300,)
(100, 768)
(100,)


**Вывод**

C помощью модели BERT, предобученной на английских текстах, были созданы эмбеддинги.

## Обучение

### Случайный лес

In [72]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=12345)

In [73]:
parametrs_rf = {'n_estimators': range (10, 101, 10),
                'min_samples_leaf': [3, 4, 5],
                'max_depth': range (1, 20, 1)}

In [74]:
grid_rf = GridSearchCV(model_rf,
                       param_grid=parametrs_rf,
                       scoring='f1',
                       cv=3,
                       n_jobs=-1)

In [75]:
grid_rf.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': range(1, 20),
                         'min_samples_leaf': [3, 4, 5],
                         'n_estimators': range(10, 101, 10)},
             scoring='f1')

In [76]:
grid_rf.best_estimator_

RandomForestClassifier(class_weight='balanced', max_depth=1, min_samples_leaf=3,
                       n_estimators=80, random_state=12345)

In [77]:
best_estim_rf = grid_rf.best_estimator_
best_estim_rf.fit(features_train, target_train)
predictions_test_rf = best_estim_rf.predict(features_test)
result_rf = f1_score(target_test, predictions_test_rf)
print("F1-мера наилучшей модели случайного леса на тестовой выборке:", result_rf)

F1-мера наилучшей модели случайного леса на тестовой выборке: 0.4444444444444445


### Логистическая регрессия

In [78]:
model_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')

In [79]:
parametrs_lr = {'fit_intercept':[True,False]}

In [80]:
grid_lr = GridSearchCV(model_lr,
                       param_grid=parametrs_lr,
                       scoring='f1',
                       cv=3,
                       n_jobs=-1)

In [81]:
grid_lr.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=12345,
                                          solver='liblinear'),
             n_jobs=-1, param_grid={'fit_intercept': [True, False]},
             scoring='f1')

In [82]:
best_estim_lr = grid_lr.best_estimator_
best_estim_lr.fit(features_train, target_train)
predictions_test_lr = best_estim_lr.predict(features_test)
result_lr = f1_score(target_test, predictions_test_lr)
print("F1-мера наилучшей модели линейной регрессии на тестовой выборке:", result_lr)

F1-мера наилучшей модели линейной регрессии на тестовой выборке: 0.75


### CatBoost

In [83]:
model_catboost = CatBoostClassifier(auto_class_weights='Balanced')

In [84]:
parametrs_catboost = {'depth': range (1, 10, 1),
                      'learning_rate': [0.01, 0.05, 0.1],
                      'iterations': range (30, 50, 10)}

In [85]:
grid_catboost = GridSearchCV(model_catboost,
                             param_grid=parametrs_catboost,
                             scoring='f1',
                             cv=3,
                             n_jobs=-1)

In [86]:
grid_catboost.fit(features_train, target_train, verbose=False)

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f7801500110>,
             n_jobs=-1,
             param_grid={'depth': range(1, 10), 'iterations': range(30, 50, 10),
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='f1')

In [87]:
best_estim_catboost = grid_catboost.best_estimator_
best_estim_catboost.fit(features_train, target_train, verbose=False)
predictions_test_catboost = best_estim_catboost.predict(features_test)
result_catboost = f1_score(target_test, predictions_test_catboost)
print("F1-мера модели градиентного бустинга на основе библиотеки CatBoost на тестовой выборке:", result_catboost)

F1-мера модели градиентного бустинга на основе библиотеки CatBoost на тестовой выборке: 0.6206896551724138


### LightGBM

In [94]:
model_lgbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight='balanced')

In [95]:
parametrs_lgbm = {'max_depth': range (1, 25, 1),
                  'n_estimators': [400, 700, 1000],
                  'num_leaves': [50, 100, 200]}

In [96]:
grid_lgbm = GridSearchCV(model_lgbm,
                         param_grid=parametrs_lgbm,
                         scoring='f1',
                         cv=3,
                         n_jobs=-1,
                         verbose=1)

In [97]:
grid_lgbm.fit(features_train, target_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits


GridSearchCV(cv=3, estimator=LGBMClassifier(class_weight='balanced'), n_jobs=-1,
             param_grid={'max_depth': range(1, 25),
                         'n_estimators': [400, 700, 1000],
                         'num_leaves': [50, 100, 200]},
             scoring='f1', verbose=1)

In [98]:
grid_lgbm.best_estimator_

LGBMClassifier(class_weight='balanced', max_depth=2, n_estimators=400,
               num_leaves=50)

In [99]:
best_estim_lgbm = grid_lgbm.best_estimator_
best_estim_lgbm.fit(features_train, target_train)
predictions_test_lgbm = best_estim_lgbm.predict(features_test)
result_lgbm = f1_score(target_test, predictions_test_lgbm)
print("F1-мера модели градиентного бустинга на основе библиотеки LightGBM на тестовой выборке:", result_lgbm)

F1-мера модели градиентного бустинга на основе библиотеки LightGBM на тестовой выборке: 0.7058823529411764


## Выводы

Для ответа на вопрос исследования была изучена таблица с данными, проведена подготовка данных, а также обучены и проанализированы модели.

**В результате финального тестирования было установлено, что лучшей моделью для классифицикации комментариев на позитивные и негативные стала модель логистической регрессии. Для данной модели в результате проверки на тестовой выборке удалось достичь наибольшее значение метрики качества F1 — 0.75.**